In [2]:
from pf_simulation import PowerFactorySim as pfsim

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
MONITORED_VARIABLES = {
    '*.ElmTerm': ['m:u1', 'm:phiu'],
    '*.ElmSym': ['c:firel', 's:outofstep']
}
VAR_FOLDER = {
    '*.ElmTerm': ['magnitude','angle'],
    '*.ElmSym': ['rotor_angle', 'out_of_step']
}

#sim = PowerFactorySim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
sim = pfsim(folder_name=FOLDER_NAME,
            project_name=PROJECT_NAME,
            study_case_name=STUDY_CASE_NAME)
sim.app.Show()

In [8]:
import os
import pandas as pd

# Membuat skenario operasi
f_line = sim.app.GetCalcRelevantObjects('*.ElmLne')
f_location = [0, 20, 50, 80]
f_duration = [0.1, 0.15, 0.18, 0.2, 0.25]
load_level = [i/100 for i in range(50,115,5)]

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

Testing OPF

In [13]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    sim.ldfAnalysis()
    gen_dispatch = sim.getDispatch()

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev
0.75 load, OPF and LF running successfully
0.80 load, OPF and LF running successfully
0.85 load, OPF and LF running successfully
0.90 load, OPF and LF running successfully
0.95 load, OPF and LF running successfully
1.00 load, OPF and LF running successfully
1.05 load, OPF and LF running successfully
1.10 load, OPF and LF running successfully
